# 🧹 Data Cleaning Notebook

This notebook documents the **step-by-step cleaning process** applied to the `Uncleaned_DS_jobs.csv` dataset to produce the cleaned `Cleaned_DS_Jobs.csv`. The cleaning involves handling duplicates, missing values, feature extraction, and standardization.

In [ ]:

import pandas as pd
import numpy as np

# Display options
pd.set_option('display.max_columns', None)

# Load datasets
raw_df = pd.read_csv('data/raw/Uncleaned_DS_jobs.csv')
cleaned_df = pd.read_csv('data/processed/Cleaned_DS_Jobs.csv')

print("Raw dataset shape:", raw_df.shape)
print("Cleaned dataset shape:", cleaned_df.shape)
raw_df.head()


## Step 1: Inspect Raw Dataset

In [ ]:

raw_df.info()
raw_df.describe(include='all').T.head(15)


## Step 2: Remove Duplicates

Duplicates were removed, reducing rows from **672 → 660**.

In [ ]:

raw_df = raw_df.drop_duplicates()
print("After removing duplicates:", raw_df.shape)


## Step 3: Handle Missing Values

Check missing values per column and decide strategies (drop, fill, or mark as NA).

In [ ]:

raw_df.isnull().sum()


## Step 4: Parse Salary Field

Split salary estimates into structured fields: `min_salary`, `max_salary`, `salary_currency`, `salary_period`. 

In [ ]:

def parse_salary(salary_str):
    if pd.isnull(salary_str) or salary_str == '-1':
        return pd.Series([np.nan, np.nan, None, None])
    
    salary_str = salary_str.replace('$', '').replace('K','000')
    parts = salary_str.split()
    try:
        if '-' in parts[0]:
            min_s, max_s = parts[0].split('-')
            min_s, max_s = int(min_s), int(max_s)
        else:
            min_s = max_s = int(parts[0])
    except:
        min_s, max_s = np.nan, np.nan
    
    currency = '$'
    period = parts[-1] if len(parts) > 1 else None
    return pd.Series([min_s, max_s, currency, period])

raw_df[['min_salary','max_salary','salary_currency','salary_period']] = raw_df['Salary Estimate'].apply(parse_salary)
raw_df[['Job Title','Salary Estimate','min_salary','max_salary']].head()


## Step 5: Extract Location Fields

Split location into `city`, `state`, `country`. 

In [ ]:

raw_df[['city','state']] = raw_df['Location'].str.split(',', n=1, expand=True)
raw_df['country'] = 'USA'  # Example assumption; adjust as needed
raw_df[['Location','city','state','country']].head()


## Step 6: Normalize Job Titles

Simplify job titles into categories (e.g., Data Scientist, Data Analyst, ML Engineer).

In [ ]:

def simplify_title(title):
    title = title.lower()
    if 'data scientist' in title:
        return 'data scientist'
    elif 'analyst' in title:
        return 'data analyst'
    elif 'engineer' in title:
        return 'ml engineer'
    else:
        return 'other'

raw_df['job_simp'] = raw_df['Job Title'].apply(simplify_title)
raw_df[['Job Title','job_simp']].head()


## Step 7: Extract Seniority Level

In [ ]:

def seniority(title):
    title = title.lower()
    if 'senior' in title or 'sr' in title:
        return 'senior'
    elif 'jr' in title or 'junior' in title:
        return 'junior'
    else:
        return 'na'

raw_df['seniority'] = raw_df['Job Title'].apply(seniority)
raw_df[['Job Title','seniority']].head()


## Step 8: Save Cleaned Dataset

In [ ]:

raw_df.to_csv('data/processed/Cleaned_DS_Jobs_generated.csv', index=False)
print("Saved cleaned dataset to data/processed/Cleaned_DS_Jobs_generated.csv")
